In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.core.pylabtools import figsize


sns.set(font_scale=2)
figsize(20, 20)

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', 50)

In [2]:
figsize(20, 20)

# Exploratory Data Analysis and Preprocessing

**We start by analyzing the data at hand**

In [3]:
telco_df = pd.read_csv('telco.csv')

telco_df.describe()

FileNotFoundError: ignored

**The dataset is really small. This can impose some though choices when it comes to outliers and feature engineering.**

**Throwing out outliers is very hard in this case as the information value of each row is high with such low row numbers.**

**If we engineer too many features the column/row ration will get very big, resulting in overfitting**


In [ ]:
telco_df.head()

## Data Quality checks

**Checking the datatypes. It seems retire field should be an int, as it is a flag.**

In [ ]:
telco_df.dtypes

In [ ]:
telco_df['retire'].unique()

In [ ]:
telco_df['retire'] = telco_df['retire'].astype('int64')

**Checking for missing values. Seems all is good. Since there are no string based fields we don't have to check for question marks or empty strings in rows.** 

In [ ]:
sns.heatmap(telco_df.isnull(), cbar = False, cmap = 'viridis')

**Checking correlation matrix to see if we have any redundant columns.**

**We don't exactly have columns that are 1:1 correlated to another. However longmon and equipmon are very highly correlated, they are not exactly identical. They might come in use at feature engineering stage so I keep them.**

In [ ]:
sns.set(font_scale=1)
sns.heatmap(telco_df[telco_df.columns[:21].insert(0,'churn')].corr(), annot=True, cmap='coolwarm', vmin=-1)

In [ ]:
sns.heatmap(telco_df[telco_df.columns[21:]].corr(), annot=True, cmap='coolwarm', vmin=-1)

**As we can see the dataset is imbalanced towards no churn. We will have to keep this in mind to avoid our models overfitting on the imbalance.**

In [ ]:
telco_df['churn'].value_counts()

## Feature Engineering

**Let's see the distributions of the columns to get some ideas.**

In [ ]:
f, axes_array = plt.subplots(len(telco_df.columns), figsize=(15,15 * len(telco_df.columns)))

for i, column in enumerate(telco_df.columns.drop('churn')):
    g = sns.FacetGrid(telco_df, hue="churn", )

    g = g.map(sns.distplot, column, rug=True, ax=axes_array[i])

In [ ]:
telco_df['employ'].value_counts()[0]
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'employ', rug=True)

**Does this mean these customers are unemployed, or that they have just switched employer?**

**Let's see if we can find different patterns in churn for them.**

In [ ]:
def is_not_zero(value):
    if value > 0:
        return 1
    return 0

telco_df['is_employed'] = telco_df['employ'].apply(is_not_zero)
telco_df[['is_employed', 'churn']].groupby('is_employed').mean()

**It seems it is worth keeping this field as this flag (whether our employment assumption is true or not) seems to highly correlate with churn**

In [ ]:
print(telco_df['tollmon'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'tollmon', rug=True)

**_tollmon_ shows a normal distribution except for the many zeros. Might worth adding a flag for this.**

In [ ]:
telco_df['has_tollfree'] = telco_df['tollmon'].apply(is_not_zero)
telco_df[['has_tollfree', 'churn']].groupby('has_tollfree').mean()

**It seems this field doesn't mean much of a difference in churn regards, we drop this field**

In [ ]:
telco_df = telco_df.drop('has_tollfree', axis=1)

**We will do the same checks for _equipmon_, _cardmon_, _wiremon_, _tollten_, _equipten_, _cardten_, _wireten_**

In [ ]:
print(telco_df['equipmon'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'equipmon', rug=True)
telco_df['has_equipment'] = telco_df['equipmon'].apply(is_not_zero)
telco_df[['has_equipment', 'churn']].groupby('has_equipment').mean()

In [ ]:
print(telco_df['cardmon'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'cardmon', rug=True)
telco_df['has_card'] = telco_df['cardmon'].apply(is_not_zero)
telco_df[['has_card', 'churn']].groupby('has_card').mean()

In [ ]:
print(telco_df['wiremon'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'wiremon', rug=True)
telco_df['has_wireless'] = telco_df['wiremon'].apply(is_not_zero)
telco_df[['has_wireless', 'churn']].groupby('has_wireless').mean()

In [ ]:
print(telco_df['tollten'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'tollten', rug=True)
telco_df['has_tollten'] = telco_df['tollten'].apply(is_not_zero)
telco_df[['has_tollten', 'churn']].groupby('has_tollten').mean()

In [ ]:
telco_df = telco_df.drop('has_tollten', axis=1)
print(telco_df['equipten'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'equipten', rug=True)
telco_df['has_equipten'] = telco_df['equipten'].apply(is_not_zero)
telco_df[['has_equipten', 'churn']].groupby('has_equipten').mean()

In [ ]:
print(telco_df['cardten'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'cardten', rug=True)
telco_df['has_cardten'] = telco_df['cardten'].apply(is_not_zero)
telco_df[['has_cardten', 'churn']].groupby('has_cardten').mean()

In [ ]:
print(telco_df['wireten'].value_counts()[0])
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'wireten', rug=True)
telco_df['has_wireten'] = telco_df['wireten'].apply(is_not_zero)
telco_df[['has_wireten', 'churn']].groupby('has_wireten').mean()

**Let's check if the newly introduced fields are redundant.**

In [ ]:
sns.set(font_scale=1.5)

df_new_fields = telco_df[['has_wireten', 'has_cardten', 'has_equipten', 'has_wireless', 'has_card', 'has_equipment', 'is_employed', 'churn']]
sns.heatmap(df_new_fields.corr(), annot=True, cmap='coolwarm', vmin=-1)

**It seems the fields we derived from over tenure usage and basic usage contain the same information, so we can drop one of them.**

In [ ]:
telco_df = telco_df.drop(['has_wireten', 'has_cardten', 'has_equipten'], axis=1)

### Binning

**Sometimes it is worth doing binning of interval data to make it more simple and understandable for man and machine alike**

**Looking at the data we see 2 potential columns for binning: tenure and age**

In [ ]:
telco_df['tenure_years'] = telco_df['tenure'] / 12
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'tenure_years', rug=True)

In [ ]:
def age_binner(age):
    if age < 25:
        return 1
    if age < 36:
        return 2
    if age < 45:
        return 3
    if age < 65:
        return 4
    return 5

telco_df['age_group'] = telco_df['age'].apply(age_binner)
g = sns.FacetGrid(telco_df, hue='churn', height=20)
g = g.map(sns.distplot, 'age_group', rug=True)

In [ ]:
df_new_fields = telco_df[['tenure', 'tenure_years', 'age', 'age_group', 'churn']]
sns.heatmap(df_new_fields.corr(), annot=True, cmap='coolwarm', vmin=-1)

**It seems binning didn't really help in this case, so we drop these columns**

In [ ]:
telco_df = telco_df.drop(['tenure_years', 'age_group'], axis=1)

### One-Hot encoding

Numerical nominal fields are to be seperated into indicator fields for each possible values. Leaving a numerical nominal field as it is is wrong, because some models are distance based and assume ordinality between the numbers. 

In some cases this can work, for example a in field _settlement type_ we could assume ordinality between settlment types going from smaller to larger. 

In the case of _region_ I assume there is no such ordinality, so I will use One-Hot Encoding for it.

In [ ]:
telco_df = telco_df.join(pd.get_dummies(telco_df['region'], prefix='region'), how='inner')
telco_df = telco_df.drop('region', axis=1)
telco_df.head()

## Data Transformation

**We prepare the data for modeling now.**

**First off we have to handle outliers of our dataset. The problem with massive outliers is that some models (like regression) can be thrown off by them.**


**The easiest way to handle outliers is to throw the row off where we see an outlier in one of the columns. Now the problem is that our dataset is very small so we can't really afford to throw out any of them. Even if there is an outlier in one of the columns of a row, the rest of the columns contain valuable data. So what we will do instead is transform the outlier values and keep the row.**


**In such a low row count we can't be sure if an outlier really is an outstanding value or just and undersampled portion of the real data. This is why I will be very forgiving with outliers.**

In [ ]:
big_columns = ['income', 'longten', 'tollten', 'equipten', 'cardten', 'wireten']
small_columns = ['tenure', 'age', 'address', 'employ', 'longmon', 'tollmon', 'equipmon', 'cardmon', 'wiremon']

fig = sns.boxplot(data=telco_df[big_columns], palette="Set2")
fig.set_xticklabels(fig.get_xticklabels(), rotation=-90)

In [ ]:
fig = sns.boxplot(data=telco_df[small_columns], palette="Set2")
fig.set_xticklabels(fig.get_xticklabels(), rotation=-90)

**In those columns where we saw heavy outliers I will take the top 5% of the values and lower them to 95% quantile.**

In [ ]:
cols_with_outliers = ['cardten', 'cardmon', 'longmon', 'longten', 'tollmon', 'tollten', 'wireten', 'wiremon']


for col in cols_with_outliers:
    q95 = telco_df[col].quantile(.95)
    telco_df[col] = telco_df[col].apply(lambda val: min(val,q95))

In [ ]:
fig = sns.boxplot(data=telco_df[big_columns], palette="Set2")
fig.set_xticklabels(fig.get_xticklabels(), rotation=-90)

In [ ]:
fig = sns.boxplot(data=telco_df[small_columns], palette="Set2")
fig.set_xticklabels(fig.get_xticklabels(), rotation=-90)

**These boxplots do indeed seem a lot friendlier after this small adjustment to the heavy outliers.**

## Normalization 

**A lot of the algorithms require normalization to give the best possible results. Without normalization KNN for example could give more weight to features with higher magnitude of values when calculating distances regardless of the correlation with the target variable.** 

In [ ]:
telco_df.describe()

In [ ]:
cols_to_be_normalized = ['tenure', 'age', 'address', 'income', 'ed', 'employ', 'reside', 'longmon', 'tollmon', 'equipmon',\
                        'cardmon', 'wiremon', 'longten', 'tollten', 'equipten', 'cardten', 'wireten', 'custcat']


for col in cols_to_be_normalized:
    telco_df[col] = telco_df[col].apply(lambda x: (x - telco_df[col].min()) / 
                (telco_df[col].max() - telco_df[col].min()) )

In [ ]:
telco_df.describe()

**Let's see the correlation values now that the we are done with data massaging**

In [ ]:
correlations_with_target = telco_df.corr()['churn'].values.reshape((-1,1))
column_names = telco_df.columns

sns.heatmap(correlations_with_target, annot=True, cmap='coolwarm', vmin=-1, yticklabels = column_names, xticklabels='')

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import sklearn
sklearn.__version__

**For modeling I will try different models with parameter optimization. I will try 3 things to compensate for the class imbalance:**

1. The scoring metric used will be F1 macro average, this will optimize for a good balance between precision and recall for both classes.
2. Will try to rebalance the dataset using Synthetic Minority Oversampling Technique (SMOTE).
3. Some algorithms can weight the samples to counteract class imbalances, will try this as well.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from tqdm import tqdm

In [ ]:
X = telco_df.drop('churn', axis=1)
y = telco_df['churn']


def param_optimization(model_class, params, k_range, k_selector=chi2, balancing=None, scoring='f1_macro'):
    best_score = -1
    best_params = None

    for k in tqdm(k_range):
        filter_kbest = SelectKBest(k_selector, k=k)
        X_new = filter_kbest.fit_transform(telco_df.drop('churn', axis=1), telco_df['churn'])

        k_best_cols = telco_df.drop('churn', axis=1).columns[filter_kbest.get_support()]
        X_train, X_valid, y_train, y_valid \
        = train_test_split(telco_df[k_best_cols], telco_df['churn'], test_size = 0.2, random_state = 42)
        
        if balancing == 'SMOTE':
            over = SMOTE(sampling_strategy=.5)
            under = RandomUnderSampler(sampling_strategy=0.4)
            pipeline = Pipeline(steps=[('under', under), ('over', over)])
            X_train, y_train = over.fit_resample(X_train, y_train)

        param_optimizer = GridSearchCV(estimator = model_class(), param_grid = params, 
                                       verbose=0, n_jobs = 4, scoring=scoring)
        param_optimizer.fit(X_train, y_train)
        
        
        model = model_class(**param_optimizer.best_params_)
        
        if balancing == 'WEIGHTED':
            sample_weights = [1 / (y_train.value_counts()[label] / len(y_train)) for label in y_train]            
            model.fit(X_train, y_train, sample_weight = sample_weights)
        else:
            model.fit(X_train, y_train)

        predictions = model.predict(X_valid)

        f1_macro_score = f1_score(y_valid, predictions, average='macro')
        if f1_macro_score > best_score:
            best_params = {'k': k, 'k_best_cols': k_best_cols}
            best_params['model_params'] = param_optimizer.best_params_
            best_score = f1_macro_score
            print(f'New Best score found! score: {best_score}, params: {best_params}')

    print(f'FINISHED OPTIMIZATION! score: {best_score}, params: {best_params}')
    return best_params

In [ ]:
best_params = param_optimization(KNeighborsClassifier, { 'n_neighbors': range(1, 2 * len(X.columns)), 'p':[1,2]},\
                                 range(10,25), balancing = 'SMOTE')


X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_knn = KNeighborsClassifier(**best_params['model_params'])
model_knn.fit(X_train, y_train)

predictions = model_knn.predict(X_valid)

print(classification_report(y_valid,predictions))
print(accuracy_score(y_valid, predictions))
print(balanced_accuracy_score(y_valid, predictions))

In [ ]:
best_params = param_optimization(KNeighborsClassifier, { 'n_neighbors': range(1, 2 * len(X.columns)), 'p':[1,2]}, range(10,25))


X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_knn = KNeighborsClassifier(**best_params['model_params'])
model_knn.fit(X_train, y_train)

predictions = model_knn.predict(X_valid)

print(classification_report(y_valid,predictions))
print(accuracy_score(y_valid, predictions))
print(balanced_accuracy_score(y_valid, predictions))

### KNN

**The KNN algorithm is sensitive to outliers and needs data to be normalized. I did this during the preprocessing so the algorithm can be used.**

**KNN doesn't use weighing of samples so I tried SMOTE. Unfortunately the rebalancing was not successful, maybe because of the low sample size the algorithm can't create good new samples for the churn samples, and throwing away non-churn samples is very costly regarding information.**

**Best KNN model statistics> F1 macro average: 0.74, Balanced Accuracy: 0.73**

**Best hyperparameters:**

- Columns: 
```python
['tenure', 'age', 'address', 'ed', 'employ', 'retire', 'equip',
       'callcard', 'wireless', 'longmon', 'equipmon', 'cardmon', 'longten',
       'tollten', 'cardten', 'voice', 'pager', 'internet', 'ebill',
       'has_equipment', 'has_card', 'has_wireless']```
- N Neighbours: 43
- Distance metric: Manhattan

In [ ]:
best_params = param_optimization(RandomForestClassifier, { 'n_estimators': range(1,50,2), 'max_depth': range(1,35,2)}\
                                 ,range(10,25), balancing = 'WEIGHTED')


X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_rf = RandomForestClassifier(**best_params['model_params'])

sample_weights = [1 / (y_train.value_counts()[label] / len(y_train)) for label in y_train]            
model_rf.fit(X_train, y_train, sample_weight = sample_weights)

predictions = model_rf.predict(X_valid)


print(classification_report(y_valid,predictions))
print(accuracy_score(y_valid, predictions))
print(balanced_accuracy_score(y_valid, predictions))

In [ ]:
best_params = param_optimization(RandomForestClassifier, { 'n_estimators': range(1,50,2), 'max_depth': range(1,35,2)},\
                                range(10,25))


X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_rf = RandomForestClassifier(**best_params['model_params'])
model_rf.fit(X_train, y_train)

predictions = model_rf.predict(X_valid)


print(classification_report(y_valid,predictions))
print(accuracy_score(y_valid, predictions))


print(balanced_accuracy_score(y_valid, predictions))

### Random Forest Classifier

**Tree based algorithms are very robust. They are almost ignorant to outliers and does not need data to be normalized.**

**Random Forests support weighing of samples and indeed turning it on helped the learning process**

**Best Random Forest model statistics> F1 macro average: 0.74, Balanced Accuracy: 0.72**

**Best hyperparameters:**

- Columns: 
```python
['tenure', 'age', 'address', 'ed', 'employ', 'retire', 'equip',
       'callcard', 'wireless', 'longmon', 'equipmon', 'cardmon', 'longten',
       'tollten', 'cardten', 'voice', 'pager', 'internet', 'ebill',
       'has_equipment', 'has_card', 'has_wireless']```
- Max Depth: 7
- N Estimators (number of trees): 19

In [ ]:
param_grid = [
    {
        'kernel': ['linear'],
        'C': np.linspace(.5, 6, 7),
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['poly'],
        'C': np.linspace(.5, 6, 7),
        'degree': range(5, 10),
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['rbf'],
        'C': np.linspace(.5, 6, 7),
        'gamma': np.linspace(0.001, 2, 10),
        'class_weight': [None, 'balanced']
    }
]

best_params = param_optimization(SVC, param_grid, [10, 13, 15, 17, 22, 25])


X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_svc = SVC(**best_params['model_params'])
model_svc.fit(X_train, y_train)

predictions = model_svc.predict(X_valid)


print(classification_report(y_valid,predictions))
print(accuracy_score(y_valid, predictions))
print(balanced_accuracy_score(y_valid, predictions))

### Support Vector Machines

**Support vector based models are also reliant on distance calculations thus sensitive to outliers and normalization.**

**SVC also supports weighing of samples but it doesn't seem to make a significant difference. The best classifier seems to be the rbf version.**

**Best SVC model statistics> F1 macro average: 0.71, Balanced Accuracy: 0.69**

**Best hyperparameters:**

- Columns: 
```python
['tenure', 'age', 'address', 'ed', 'employ', 'retire', 'equip',
       'callcard', 'wireless', 'longmon', 'equipmon', 'cardmon', 'longten',
       'tollten', 'cardten', 'voice', 'pager', 'internet', 'ebill',
       'has_equipment', 'has_card', 'has_wireless']```
- kernel: rbf
- C: 6
- gamma: 0.22


Note: On my machine SVC hyperparameter optimization failed multiple times and had to be restarted, in case I had access to a stronger machine I am sure this would yield the best results.

# Summary

**The given model contained very few samples for reliable churn prediction, thus I had to be very gentle with outliers.**

**The dataset is imbalanced and I could hardly create good synthetic data due to the low sample count.**

**The dataset contained a few columns where I could extract additional information by feature engineering. The engineered *has_card*, *has_equipment* and *has_wireless* columns are among the best K features selected.**

**I would advise adding 2 additional columns to the data:**
- **competition_investment: Has there been any infrastructural investments made by the competition in the past 6 months**
- **complaints: Has the subscriber called the service desk due to outages or any other complaints regarding the service in the past 3 months? This could even be a numeric field containing the number of calls, or the last call date.**

**The best model I created was KNN with a balanced accuracy of 73%. Below we can see the confusion matrix of it on the validation set.**

**In the future I would definetily try building a neural network model for the data. Also I would try a more extensive hyperparameter tuning on a stronger machine, along with a PCA dimensionality reduction.**


In [ ]:
X_train, X_valid, y_train, y_valid \
    = train_test_split(telco_df[best_params['k_best_cols']], telco_df['churn'], test_size = 0.2, random_state = 42)

model_knn.fit(X_train, y_train)

predictions = model_knn.predict(X_valid)

conf_matrix = confusion_matrix(y_valid, predictions)


sns.set(font_scale=2)
figsize(10, 10)

fig = sns.heatmap(conf_matrix, annot=True, linewidths=.5, cmap='coolwarm', cbar=False, fmt='d')
fig.set_ylabel('Actual')
fig.set_xlabel('Predicted')